# TCG Shadowbox Generator - Quick Start

このノートブックは **ALL RUN** で一気に実行できるシンプルなクイックスタートです。

**使い方:**
1. 下の「設定セクション」で変数を設定
2. メニューから **Run > Run All Cells** を実行
3. 結果を確認

> 詳細な実験や各検出メソッドの比較は `02_detailed_experiments.ipynb` を参照してください。

---
## 設定セクション

以下の変数を設定してください。

In [ ]:
# =============================================================================
# 設定変数（ここを編集してください）
# =============================================================================

# --- 3Dモデル生成モード ---
# "depth": 深度推定+クラスタリング（デフォルト、シャドーボックス風レイヤー）
# "triposr": TripoSRによる直接3Dメッシュ生成（要: pip install shadowbox[triposr]）
MODEL_MODE = "depth"

# --- 画像ソース ---
# 画像の読み込み方法を選択: "url", "file", "directory"
IMAGE_MODE = "directory"

# URL指定の場合
IMAGE_URL = "https://example.com/your-card-image.png"

# ファイルパス指定の場合
IMAGE_FILE = "../data/cards/your-card.png"

# ディレクトリ指定の場合（ギャラリーから選択）
IMAGE_DIR = "../data/cards/"
IMAGE_INDEX = 0  # 選択する画像のインデックス（0から始まる）

# --- イラスト領域検出 ---
# 検出方法: "auto", "manual", または特定のメソッド名
# 利用可能なメソッド:
#   "auto"                    - 自動（最も信頼度の高い結果を使用）
#   "gradient_richness"       - 勾配の豊かさ（おすすめ）
#   "edge_detection"          - エッジ検出
#   "hsv_threshold"           - HSV閾値
#   "contour_detection"       - 輪郭検出
#   "grid_scoring"            - グリッドスコアリング
#   "boundary_contrast"       - 境界コントラスト
#   "frame_analysis"          - フレーム解析
#   "band_complexity"         - 水平帯複雑度
#   "horizontal_lines"        - 水平線検出
#   "center_expansion"        - 中央拡張
#   "manual"                  - 手動指定（MANUAL_BBOX を設定）
DETECTION_METHOD = "gradient_richness"

# 手動指定の場合のバウンディングボックス (x, y, width, height)
MANUAL_BBOX = (59, 133, 375, 379)

# --- 処理モード ---
# True: 生深度モード（クラスタリングなし、滑らかな深度）
# False: クラスタリングモード（離散的なレイヤー、シャドーボックス感強い）
USE_RAW_DEPTH = False

# クラスタリングモード時のレイヤー数（Noneで自動探索）
NUM_LAYERS = 5

# 生深度モード時の深度スケール（大きいほど立体感が増す）
DEPTH_SCALE = 1.5

# --- レイヤーモード ---
# True: 累積モード（奥のレイヤーが手前を継承、ポイント多い）
# False: 穴あきモード（各レイヤーは自分のピクセルのみ、ポイント少ない）
CUMULATIVE_LAYERS = True

# --- 背面パネル ---
# True: 最背面にカード全体画像を追加（シャドーボックスの背景）
# False: 背面パネルなし
BACK_PANEL = True

# --- レイヤー補間 ---
# レイヤー間に挿入する補間レイヤーの数（0で無効）
# フレーム(レイヤー0)は最背面まで、他のレイヤーは次のレイヤーまで補間
# 本物のシャドーボックスのようにフレームが壁として続く効果
LAYER_INTERPOLATION = 2

# --- レイヤー飛び出し ---
# イラストレイヤーをフレームより手前に飛び出させる量（0.0〜1.0）
# フレームの厚み（最前面から最背面までの距離）に対する比率
# 0.0: 飛び出しなし、0.3: 30%飛び出し、1.0: フレームと同じ厚み分飛び出し
LAYER_POP_OUT = 0.7

# --- レイヤー間隔モード ---
# "even": 均等配置（デフォルト、レイヤーを等間隔に配置）
# "proportional": 比例配置（centroidの深度値に比例した間隔で配置）
#   centroidが [0.05, 0.10, 0.80, 0.95] の場合:
#     even:         均等 → z = [-0.125, -0.250, -0.375, -0.500]
#     proportional: 比例 → z = [-0.026, -0.053, -0.421, -0.500]
LAYER_SPACING_MODE = "proportional"

# --- レイヤーマスクモード ---
# "cluster": クラスタラベルでレイヤー形状を決定（デフォルト、平板レイヤー）
# "contour": 生深度の等高線でレイヤー形状を切り取り（等高線カット）
LAYER_MASK_MODE = "contour"

# --- 額縁の厚み ---
# シャドーボックス全体の厚み（固定値）
# レイヤー数に関係なく一定の厚みになる
FRAME_DEPTH = 0.3

# --- パフォーマンス設定 ---
# 最大解像度（ピクセル）。大きい画像を縮小してレンダリングを高速化。
# None: 縮小なし, 200: 軽量プレビュー, 400: バランス, 800: 高品質
MAX_RESOLUTION = 200  # おすすめ: プレビューは200、最終出力はNone

# --- 表示オプション ---
# カードフレームを含めるか（True: 本物のシャドーボックス風）
INCLUDE_CARD_FRAME = True

# 3Dフレーム（額縁）を表示するか
SHOW_FRAME = True

# --- 3D描画モード ---
# "points": ポイント描画（軽量、従来方式）
# "mesh": メッシュ描画（Blender風のきれいな面、重い）
RENDER_MODE = "mesh"

# --- エクスポート設定 ---
# エクスポートを有効にするか
EXPORT_ENABLED = False
EXPORT_PATH = "../data/output/shadowbox"  # 出力ディレクトリ

# --- モックモード（テスト用） ---
# True: モック深度推定（高速、モデル不要）
# False: 実際の深度推定（初回はモデルダウンロードあり）
USE_MOCK_DEPTH = False

print("設定完了!")

---
## セットアップ

In [ ]:
# インポート
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

from shadowbox import create_pipeline
from shadowbox.config import BoundingBox, ShadowboxSettings
from shadowbox.detection import RegionDetector, detect_illustration_region
from shadowbox.gui import ImageSelector, load_from_url
from shadowbox.utils.image import load_image
from shadowbox.visualization import (
    create_depth_heatmap,
    show_clustering_summary,
    render_shadowbox,
    RenderOptions,
    export_to_stl,
    export_to_obj,
    export_to_ply,
)

# matplotlibの日本語フォント設定（Windows）
import matplotlib
matplotlib.rcParams['font.family'] = ['MS Gothic', 'Yu Gothic', 'Meiryo', 'sans-serif']
matplotlib.rcParams['axes.unicode_minus'] = False

print("セットアップ完了!")

---
## 1. 画像読み込み

In [ ]:
# 画像を読み込み
if IMAGE_MODE == "url":
    print(f"URLから読み込み: {IMAGE_URL}")
    image = load_from_url(IMAGE_URL)
elif IMAGE_MODE == "file":
    print(f"ファイルから読み込み: {IMAGE_FILE}")
    image = load_image(IMAGE_FILE)
elif IMAGE_MODE == "directory":
    print(f"ディレクトリから読み込み: {IMAGE_DIR}")
    selector = ImageSelector(IMAGE_DIR)
    selector.show_gallery()  # ギャラリー表示
    image = selector.select(IMAGE_INDEX)
    print(f"選択: [{IMAGE_INDEX}] {selector.image_paths[IMAGE_INDEX].name}")
else:
    raise ValueError(f"未知のIMAGE_MODE: {IMAGE_MODE}")

print(f"画像サイズ: {image.size[0]} x {image.size[1]}")

# 画像を表示
plt.figure(figsize=(6, 8))
plt.imshow(image)
plt.title("入力画像")
plt.axis("off")
plt.show()

---
## 2. イラスト領域検出

In [ ]:
# イラスト領域を検出
if DETECTION_METHOD == "manual":
    # 手動指定
    bbox = BoundingBox(
        x=MANUAL_BBOX[0],
        y=MANUAL_BBOX[1],
        width=MANUAL_BBOX[2],
        height=MANUAL_BBOX[3],
    )
    detection_method = "manual"
    confidence = 1.0
elif DETECTION_METHOD == "auto":
    # 自動検出（最良の結果を使用）
    result = detect_illustration_region(image)
    bbox = result.bbox
    detection_method = result.method
    confidence = result.confidence
else:
    # 特定のメソッドを使用
    detector = RegionDetector(min_area_ratio=0.05, max_area_ratio=0.95)
    cv_image = detector._pil_to_cv(image)
    method_map = {
        "gradient_richness": detector._detect_by_gradient_richness,
        "edge_detection": detector._detect_by_edges,
        "hsv_threshold": detector._detect_by_hsv_threshold,
        "contour_detection": detector._detect_by_contours,
        "grid_scoring": detector._detect_by_grid_scoring,
        "boundary_contrast": detector._detect_by_boundary_contrast,
        "frame_analysis": detector._detect_by_frame_analysis,
        "band_complexity": detector._detect_by_band_complexity,
        "horizontal_lines": detector._detect_by_horizontal_lines,
        "center_expansion": detector._detect_by_center_expansion,
    }
    if DETECTION_METHOD in method_map:
        result = method_map[DETECTION_METHOD](cv_image)
        if result:
            bbox = result.bbox
            detection_method = result.method
            confidence = result.confidence
        else:
            print(f"警告: {DETECTION_METHOD} では検出できませんでした。自動検出にフォールバック。")
            result = detect_illustration_region(image)
            bbox = result.bbox
            detection_method = result.method
            confidence = result.confidence
    else:
        print(f"警告: 未知のメソッド '{DETECTION_METHOD}'。自動検出を使用。")
        result = detect_illustration_region(image)
        bbox = result.bbox
        detection_method = result.method
        confidence = result.confidence

print(f"検出方法: {detection_method}")
print(f"信頼度: {confidence:.2f}")
print(f"領域: x={bbox.x}, y={bbox.y}, w={bbox.width}, h={bbox.height}")

# 検出結果を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# 元画像と検出領域
axes[0].imshow(image)
rect = plt.Rectangle(
    (bbox.x, bbox.y), bbox.width, bbox.height,
    linewidth=3, edgecolor='lime', facecolor='none'
)
axes[0].add_patch(rect)
axes[0].set_title(f"検出結果: {detection_method}\n信頼度: {confidence:.2f}")
axes[0].axis("off")

# クロップ領域プレビュー
cropped_preview = image.crop((bbox.x, bbox.y, bbox.x + bbox.width, bbox.y + bbox.height))
axes[1].imshow(cropped_preview)
axes[1].set_title(f"イラスト領域\n({bbox.width} x {bbox.height} px)")
axes[1].axis("off")

plt.tight_layout()
plt.show()

---
## 3. 深度推定 & メッシュ生成

In [ ]:
# 設定を作成
settings = ShadowboxSettings()
settings.model_mode = MODEL_MODE
settings.render.cumulative_layers = CUMULATIVE_LAYERS
settings.render.back_panel = BACK_PANEL
settings.render.layer_interpolation = LAYER_INTERPOLATION
settings.render.layer_pop_out = LAYER_POP_OUT
settings.render.frame_depth = FRAME_DEPTH
settings.render.layer_spacing_mode = LAYER_SPACING_MODE
settings.render.layer_mask_mode = LAYER_MASK_MODE

# パイプラインを作成
pipeline = create_pipeline(settings=settings, use_mock_depth=USE_MOCK_DEPTH)

# 処理を実行
if MODEL_MODE == "triposr":
    # TripoSRモード（共通パス: 深度復元 → MeshGenerator）
    print("TripoSRモードで処理中...")
    if USE_RAW_DEPTH:
        print("※ USE_RAW_DEPTH=True のため NUM_LAYERS は無視されます")
    result = pipeline.process(
        image,
        bbox=bbox,
        k=NUM_LAYERS,
        include_frame=SHOW_FRAME,
        include_card_frame=INCLUDE_CARD_FRAME,
        use_raw_depth=USE_RAW_DEPTH,
        depth_scale=DEPTH_SCALE,
        max_resolution=MAX_RESOLUTION,
    )

    mode_name = "生深度モード" if USE_RAW_DEPTH else f"クラスタリングモード (k={result.optimal_k})"
    layer_mode = "累積モード" if CUMULATIVE_LAYERS else "穴あきモード"
    back_panel_mode = "あり" if BACK_PANEL else "なし"
    interp_mode = f"{LAYER_INTERPOLATION}段" if LAYER_INTERPOLATION > 0 else "なし"
    pop_out_mode = f"{LAYER_POP_OUT*100:.0f}%" if LAYER_POP_OUT > 0 else "なし"
    spacing_mode = "比例配置" if LAYER_SPACING_MODE == "proportional" else "均等配置"
    mask_mode = "等高線カット" if LAYER_MASK_MODE == "contour" else "クラスタマスク"
    print(f"処理モード: TripoSR → {mode_name}")
    print(f"レイヤーモード: {layer_mode}")
    print(f"レイヤーマスク: {mask_mode}")
    print(f"背面パネル: {back_panel_mode}")
    print(f"レイヤー補間: {interp_mode}")
    print(f"レイヤー飛び出し: {pop_out_mode}")
    print(f"レイヤー間隔: {spacing_mode}")
    print(f"額縁の厚み: {FRAME_DEPTH}")
    print(f"レイヤー数: {result.mesh.num_layers}")
    print(f"総頂点数: {result.mesh.total_vertices:,}")
    if result.depth_map is not None:
        print(f"深度範囲: {result.depth_map.min():.3f} ~ {result.depth_map.max():.3f}")
else:
    # 深度推定モード（デフォルト）
    result = pipeline.process(
        image,
        custom_bbox=bbox,
        k=NUM_LAYERS,
        include_frame=SHOW_FRAME,
        include_card_frame=INCLUDE_CARD_FRAME,
        use_raw_depth=USE_RAW_DEPTH,
        depth_scale=DEPTH_SCALE,
        max_resolution=MAX_RESOLUTION,
    )

    mode_name = "生深度モード" if USE_RAW_DEPTH else f"クラスタリングモード (k={result.optimal_k})"
    layer_mode = "累積モード" if CUMULATIVE_LAYERS else "穴あきモード"
    back_panel_mode = "あり" if BACK_PANEL else "なし"
    interp_mode = f"{LAYER_INTERPOLATION}段" if LAYER_INTERPOLATION > 0 else "なし"
    pop_out_mode = f"{LAYER_POP_OUT*100:.0f}%" if LAYER_POP_OUT > 0 else "なし"
    spacing_mode = "比例配置" if LAYER_SPACING_MODE == "proportional" else "均等配置"
    mask_mode = "等高線カット" if LAYER_MASK_MODE == "contour" else "クラスタマスク"
    print(f"処理モード: {mode_name}")
    print(f"レイヤーモード: {layer_mode}")
    print(f"レイヤーマスク: {mask_mode}")
    print(f"背面パネル: {back_panel_mode}")
    print(f"レイヤー補間: {interp_mode}")
    print(f"レイヤー飛び出し: {pop_out_mode}")
    print(f"レイヤー間隔: {spacing_mode}")
    print(f"額縁の厚み: {FRAME_DEPTH}")
    print(f"処理後サイズ: {result.cropped_image.shape[1]} x {result.cropped_image.shape[0]}")
    print(f"レイヤー数: {result.mesh.num_layers}")
    print(f"総頂点数: {result.mesh.total_vertices:,}")
    print(f"深度範囲: {result.depth_map.min():.3f} ~ {result.depth_map.max():.3f}")

---
## 4. 2D可視化（深度ヒートマップ & クラスタリングサマリー）

In [ ]:
# 深度ヒートマップ（depth_mapがある場合に表示）
if hasattr(result, 'depth_map') and result.depth_map is not None:
    cropped_img = result.cropped_image if result.cropped_image is not None else result.original_image
    fig, axes = create_depth_heatmap(
        result.depth_map,
        cropped_img,
        cmap="viridis",
        title="深度推定結果",
    )
    plt.show()
else:
    print("深度マップが利用できません（depth_to_meshが未設定のTripoSRモード）。")

In [ ]:
# クラスタリングサマリー（クラスタリングモードの場合に表示）
if USE_RAW_DEPTH:
    print("生深度モードではクラスタリングサマリーはスキップされます。")
elif (
    hasattr(result, 'labels') and result.labels is not None
    and hasattr(result, 'centroids') and result.centroids is not None
    and hasattr(result, 'depth_map') and result.depth_map is not None
):
    cropped_img = result.cropped_image if result.cropped_image is not None else result.original_image
    fig, axes = show_clustering_summary(
        cropped_img,
        result.depth_map,
        result.labels,
        result.centroids,
    )
    plt.show()
else:
    print("クラスタリングデータが利用できません。")

---
## 5. 3Dシャドーボックス表示

**操作方法:**
- マウス左ドラッグ: 回転
- マウス右ドラッグ: パン
- スクロール: ズーム

In [ ]:
# レンダリングオプション
options = RenderOptions(
    background_color=(30, 30, 40),
    point_size=4.0,
    mesh_size=0.016,  # メッシュモード時の四角形サイズ
    show_frame=SHOW_FRAME,
    window_size=(1000, 800),
    title="TCG Shadowbox Viewer",
)

# 3Dレンダリング
print(f"描画モード: {RENDER_MODE}")
render_shadowbox(result.mesh, options, render_mode=RENDER_MODE)

---
## 6. エクスポート（オプション）

STL/OBJ/PLY形式でエクスポートして、外部3Dビューア（Windows 3D Viewer、MeshLab等）で表示できます。

In [ ]:
# メッシュをエクスポート（EXPORT_ENABLED=True の場合のみ実行）
if EXPORT_ENABLED:
    from datetime import datetime
    
    export_dir = Path(EXPORT_PATH)
    export_dir.mkdir(parents=True, exist_ok=True)
    
    # 元ファイル名を取得
    if IMAGE_MODE == "directory":
        source_name = selector.image_paths[IMAGE_INDEX].stem  # 拡張子なし
    elif IMAGE_MODE == "file":
        source_name = Path(IMAGE_FILE).stem
    elif IMAGE_MODE == "url":
        source_name = Path(IMAGE_URL).stem or "url_image"
    else:
        source_name = "shadowbox"
    
    # タイムスタンプを生成
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # ファイル名を作成: {元ファイル名}_{日時}
    base_name = f"{source_name}_{timestamp}"
    
    # PLY形式（カラー付きポイントクラウド、MeshLab/CloudCompare向け）
    export_to_ply(result.mesh, export_dir / f"{base_name}.ply")
    
    # STL形式（3Dプリンタ/Windows 3D Viewer向け）
    export_to_stl(result.mesh, export_dir / f"{base_name}.stl")
    
    # OBJ形式（Blender等向け）
    export_to_obj(result.mesh, export_dir / f"{base_name}.obj")
    
    print(f"\nエクスポート完了:")
    print(f"  {base_name}.ply")
    print(f"  {base_name}.stl")
    print(f"  {base_name}.obj")
    print(f"\n出力先: {export_dir.absolute()}")
else:
    print("エクスポートは無効です。EXPORT_ENABLED=True で有効化できます。")

---
## 次のステップ

- **設定を変更して再実行**: 上の設定セクションを編集し、「Run All」で再実行
- **詳細な実験**: `02_detailed_experiments.ipynb` で各検出メソッドの比較や手動選択を試す
- **パラメータ調整**:
  - `MODEL_MODE`: "depth" で深度推定+クラスタリング（シャドーボックス風）、"triposr" でTripoSRによる直接3Dメッシュ生成
  - `MAX_RESOLUTION`: 200で軽量プレビュー、Noneで高品質
  - `NUM_LAYERS`: レイヤー数を増減してシャドーボックスの段数を調整
  - `USE_RAW_DEPTH`: True で滑らかな深度表現、False で段差のあるシャドーボックス風
  - `DEPTH_SCALE`: 生深度モードで立体感を調整
  - `INCLUDE_CARD_FRAME`: カード全体を含めた本物のシャドーボックス風表示
  - `CUMULATIVE_LAYERS`: True で累積モード（奥のレイヤーが手前を継承）、False で穴あきモード（各レイヤーは自分のピクセルのみ）
  - `BACK_PANEL`: True で最背面にカード全体画像を追加（シャドーボックスの背景として表示）
  - `LAYER_INTERPOLATION`: レイヤー間の補間数。フレームは最背面まで、他は次のレイヤーまで補間（本物のシャドーボックス風の壁効果）
  - `LAYER_POP_OUT`: イラストレイヤーの飛び出し量（0.0〜1.0）。フレームの厚みに対する比率で、本物のシャドーボックスのようにイラストが額縁から飛び出す効果
  - `LAYER_SPACING_MODE`: レイヤーのZ軸配置モード。"even" で均等配置（デフォルト）、"proportional" でcentroid深度値に比例した配置（深度差が大きいレイヤー間は広く、小さいレイヤー間は狭くなる）
  - `LAYER_MASK_MODE`: レイヤーのマスクモード。"cluster" でクラスタラベルによる平板レイヤー（デフォルト）、"contour" で生深度の等高線による切り抜き（手前レイヤーほど狭く、奥ほど広い等高線形状）
  - `FRAME_DEPTH`: 額縁の厚み（固定値）。レイヤー数に関係なく一定の厚みになる
  - `RENDER_MODE`: "points" で軽量ポイント描画、"mesh" でBlender風のきれいなメッシュ描画
- **TripoSRモードを使用するには**:
  - 依存関係をインストール: `pip install shadowbox[triposr]`
  - `MODEL_MODE = "triposr"` に設定
  - 注意: TripoSRはGPU推奨（CPUでも動作するが遅い）
- **エクスポート**:
  - `EXPORT_ENABLED=True` でSTL/OBJ/PLY形式にエクスポート
  - Windows 3D Viewer、MeshLab、Blender等で開けます